In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nba_api.stats.static import teams

df_all_stars = pd.DataFrame()
for year in range(2014,2026):
    url = f"https://basketball.realgm.com/nba/allstar/game/rosters/{year}"

    # Get the page content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find tables with the specific Bootstrap classes
    tables = soup.find_all('table', class_='table table-striped table-hover table-bordered table-compact table-nowrap')

    # Get the third Bootstrap table (the combined roster table)
    if len(tables) >= 3:
        third_table = tables[2]
        # Convert to DataFrame
        df_this_year = pd.read_html(str(third_table))[0]
        df_this_year['Year'] = year
        df_all_stars = pd.concat([df_all_stars, df_this_year], ignore_index=True)
    else:
        third_table = tables[1]
        # Convert to DataFrame
        df_this_year = pd.read_html(str(third_table))[0]
        df_this_year['Year'] = year
        df_all_stars = pd.concat([df_all_stars, df_this_year], ignore_index=True)

df_all_stars = df_all_stars[['Player','Team','Year']]
df_all_stars.columns = df_all_stars.columns.str.lower()

# Get list of all NBA teams
nba_teams = teams.get_teams()

# Join NBA teams with All-Stars
df_nbateams = pd.DataFrame(nba_teams)[['full_name', 'abbreviation', 'nickname', 'city', 'state']]
df_all_stars = df_all_stars.merge(df_nbateams, how='inner', left_on='team', right_on='full_name')
df_all_stars = df_all_stars[['player','team', 'year', 'abbreviation']]
    

/tmp/ipykernel_2036/82939855.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_this_year = pd.read_html(str(third_table))[0]
/tmp/ipykernel_2036/82939855.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_this_year = pd.read_html(str(third_table))[0]
/tmp/ipykernel_2036/82939855.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_this_year = pd.read_html(str(third_table))[0]
/tmp/ipykernel_2036/82939855.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_this_year = pd.read_ht

In [2]:
df_all_stars.year.value_counts().reset_index().sort_values(by='year')

,year,count
5,2014,25
0,2015,28
3,2016,26
6,2017,25
1,2018,27
7,2019,25
11,2020,23
9,2021,25
8,2022,25
2,2023,26


In [3]:
df_all_stars

,player,team,year,abbreviation
0,LaMarcus Aldridge,Portland Trail Blazers,2014,POR
1,Carmelo Anthony,New York Knicks,2014,NYK
2,Chris Bosh,Miami Heat,2014,MIA
3,Kobe Bryant,Los Angeles Lakers,2014,LAL
4,Stephen Curry,Golden State Warriors,2014,GSW
...,...,...,...,...
300,Jayson Tatum,Boston Celtics,2025,BOS
301,Karl-Anthony Towns,New York Knicks,2025,NYK
302,Victor Wembanyama,San Antonio Spurs,2025,SAS
303,Jalen Williams,Oklahoma City Thunder,2025,OKC


In [4]:
from sqlalchemy import create_engine, text

# Create engine (file-backed SQLite)
engine = create_engine("sqlite:///analysis.db")

# Save DataFrame to table
df_all_stars.to_sql(
    "nba_all_stars_table",
    con=engine,
    if_exists="replace",
    index=False
)

# Run SQL and get DataFrame back
result = pd.read_sql_query(
    text("""
         
         WITH CTE AS (SELECT team, year, COUNT(*) AS total_all_stars
         FROM nba_all_stars_table
         GROUP BY team, year
         HAVING COUNT(*) >= 3)

         SELECT DISTINCT team, year, total_all_stars FROM CTE
         
         """),
    con=engine
)

result


,team,year,total_all_stars
0,Atlanta Hawks,2015,4
1,Brooklyn Nets,2021,3
2,Cleveland Cavaliers,2017,3
3,Cleveland Cavaliers,2025,3
4,Golden State Warriors,2016,3
5,Golden State Warriors,2017,4
6,Golden State Warriors,2018,4
7,Golden State Warriors,2019,3
8,Golden State Warriors,2022,3
9,Miami Heat,2014,3


In [5]:
result.merge(df_all_stars, how='inner', on=['team','year']).sort_values(by=['year','team'])

,team,year,total_all_stars,player,abbreviation
30,Miami Heat,2014,3,Chris Bosh,MIA
31,Miami Heat,2014,3,LeBron James,MIA
32,Miami Heat,2014,3,Dwyane Wade,MIA
0,Atlanta Hawks,2015,4,Al Horford,ATL
1,Atlanta Hawks,2015,4,Kyle Korver,ATL
2,Atlanta Hawks,2015,4,Paul Millsap,ATL
3,Atlanta Hawks,2015,4,Jeff Teague,ATL
13,Golden State Warriors,2016,3,Stephen Curry,GSW
14,Golden State Warriors,2016,3,Draymond Green,GSW
15,Golden State Warriors,2016,3,Klay Thompson,GSW


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


for year in range(2017,2024):
    url=f"https://www.foxsports.com/nba/draymond-green-player-game-log?season={year}"

    # Get the page content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find tables with the specific Bootstrap classes
    tables = soup.find_all('table', class_='table table-striped table-hover table-bordered table-compact table-nowrap')

    print(len(tables))

#     # Get the third Bootstrap table (the combined roster table)
#     if len(tables) >= 3:
#         third_table = tables[2]
#         # Convert to DataFrame
#         df_this_year = pd.read_html(str(third_table))[0]
#         df_this_year['Year'] = year
#         df_all_stars = pd.concat([df_all_stars, df_this_year], ignore_index=True)
#     else:
#         third_table = tables[1]
#         # Convert to DataFrame
#         df_this_year = pd.read_html(str(third_table))[0]
#         df_this_year['Year'] = year
#         df_all_stars = pd.concat([df_all_stars, df_this_year], ignore_index=True)

# df_all_stars = df_all_stars[['Player','Team','Year']]
# df_all_stars.columns = df_all_stars.columns.str.lower()

# # Get list of all NBA teams
# nba_teams = teams.get_teams()

# # Join NBA teams with All-Stars
# df_nbateams = pd.DataFrame(nba_teams)[['full_name', 'abbreviation', 'nickname', 'city', 'state']]
# df_all_stars = df_all_stars.merge(df_nbateams, how='inner', left_on='team', right_on='full_name')
# df_all_stars = df_all_stars[['player','team', 'year', 'abbreviation']]
    

0
0
0
0
0
0
0


In [20]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.foxsports.com/nba/draymond-green-player-game-log?season=2017"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.foxsports.com/",
}

html = requests.get(url, headers=headers, timeout=60).text
soup = BeautifulSoup(html, "html.parser")

# 1) Look for Next.js payload
script = soup.find("script", id="__NEXT_DATA__", type="application/json")
if not script:
    raise RuntimeError("No __NEXT_DATA__ JSON found; site structure may have changed.")

data = json.loads(script.string)

# 2) Explore keys to find the game log
# (print the top-level keys once to see structure)
# print(json.dumps(list(data.keys()), indent=2))

# This path is an example. Inspect the JSON to confirm or adjust:
# e.g., data["props"]["pageProps"]["dehydratedState"]["queries"][0]["state"]["data"]
def find_rows(d):
    if isinstance(d, dict):
        for k, v in d.items():
            # heuristic: look for a list of dicts with stat-like keys
            if isinstance(v, list) and v and isinstance(v[0], dict) and "GAME_DATE" in v[0] or "pts" in v[0] or "min" in v[0]:
                return v
            found = find_rows(v)
            if found is not None:
                return found
    elif isinstance(d, list):
        for item in d:
            found = find_rows(item)
            if found is not None:
                return found
    return None

rows = find_rows(data)
if rows is None:
    raise RuntimeError("Could not locate game-log rows in JSON. Inspect `data` structure.")

# 3) Normalize to a DataFrame
df = pd.json_normalize(rows)
print(df.head())


RuntimeError: No __NEXT_DATA__ JSON found; site structure may have changed.

In [22]:
player_id = players.find_players_by_full_name("Draymond Green")[0]["id"]
player_id

203110

In [25]:
import requests
import pandas as pd

API_KEY = "jE7yBJVRNAwdDesMgTzTXUUSx1It41Fq"

def fox_gamelog(player_id: int, season: int, season_type: str = "reg") -> pd.DataFrame:
    """
    season_type: 'reg' (regular season) or 'post' (postseason). 'pre' may also work.
    """
    url = f"https://api.foxsports.com/bifrost/v1/nba/player/{player_id}/gamelog-exp"
    params = {"apikey": API_KEY, "season": season, "seasonType": season_type}
    r = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=60)
    r.raise_for_status()
    data = r.json()

    # If we forgot/used wrong params, Fox returns a picker with 'items'
    if isinstance(data, dict) and "items" in data:
        raise ValueError(
            "Got a metadata response (season/seasonType list). "
            "Pass seasonType='reg' or 'post'."
        )

    # Common shape: data -> gamelog -> games (list of dicts)
    # If your payload differs, print(data.keys()) once and adjust the path.
    node = data.get("data") or data
    gamelog = node.get("gamelog") if isinstance(node, dict) else None
    games = (gamelog or {}).get("games") if isinstance(gamelog, dict) else None
    if not games:
        # fall back to first list-of-dicts anywhere
        def find_list_of_dicts(o):
            if isinstance(o, list) and o and isinstance(o[0], dict):
                return o
            if isinstance(o, dict):
                for v in o.values():
                    hit = find_list_of_dicts(v)
                    if hit is not None:
                        return hit
            return None
        games = find_list_of_dicts(data)
    if not games:
        raise RuntimeError("Couldn't find game rows in payload. Inspect the JSON to confirm path.")

    df = pd.json_normalize(games)
    return df

# Draymond Green = 1771, 2017 season, REGULAR SEASON
df = fox_gamelog(1771, 2017, season_type="reg")
print(df.head())


          name          id                                              items
0       Season      season  [{'id': '2024', 'name': '2024-25', 'webUrl': '...
1  Season Type  seasonType  [{'id': 'reg', 'name': 'REGULAR SEASON', 'webU...


In [30]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment

url = "https://www.basketball-reference.com/players/g/greendr01/gamelog/2025"
headers = {"User-Agent": "Mozilla/5.0"}  # BR blocks no-UA requests
html = requests.get(url, headers=headers, timeout=60).text
soup = BeautifulSoup(html, "lxml")

# Find the comment containing the table
wrapper = soup.find("div", id="all_pgl_basic")
if not wrapper:
    raise RuntimeError("Could not find 'all_pgl_basic' div; page structure may have changed.")

comment = None
for element in wrapper.children:
    if isinstance(element, Comment):
        comment = element
        break
if comment is None:
    raise RuntimeError("Could not find commented table HTML inside wrapper.")

# Parse the commented table
table_soup = BeautifulSoup(comment, "lxml").find("table", id="pgl_basic")
df = pd.read_html(str(table_soup))[0]

# Clean up repeated header rows and non-game entries
df = df[df["Rk"] != "Rk"].copy()
df = df[df["G"].notna()]
df.reset_index(drop=True, inplace=True)

print(df.head())


RuntimeError: Could not find 'all_pgl_basic' div; page structure may have changed.

In [31]:
pd.read_csv("draymond_2025.csv")

,Rk,Gcar,Gtm,Date,Team,Unnamed: 5,Opp,Result,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,814.0,1.0,2024-10-23,GSW,@,POR,W 140-104,*,20:26,...,6,6,3,1,2,3,3,8,8.8,12
1,2.0,815.0,2.0,2024-10-25,GSW,@,UTA,W 127-86,*,19:51,...,2,2,4,3,0,2,3,3,3.8,12
2,3.0,816.0,3.0,2024-10-27,GSW,NaN,LAC,L 104-112,*,26:49,...,5,5,2,0,0,2,4,2,-0.1,2
3,4.0,817.0,4.0,2024-10-29,GSW,NaN,NOP,W 124-106,*,28:30,...,2,3,7,1,2,0,4,3,7.6,15
4,5.0,818.0,5.0,2024-10-30,GSW,NaN,NOP,W 104-89,*,30:07,...,3,4,6,1,5,3,5,14,13.6,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,65.0,878.0,79.0,2025-04-08,GSW,@,PHO,W 133-95,*,20:22,...,2,2,4,2,0,3,1,5,4.6,19
79,66.0,879.0,80.0,2025-04-09,GSW,NaN,SAS,L 111-114,*,33:25,...,6,9,8,2,1,3,4,13,16.9,-6
80,67.0,880.0,81.0,2025-04-11,GSW,@,POR,W 103-86,*,29:12,...,7,7,5,4,1,2,2,3,6.7,13
81,68.0,881.0,82.0,2025-04-13,GSW,NaN,LAC,L 119-124 (OT),*,38:29,...,3,3,4,1,2,2,5,14,11.8,-5


In [34]:
from nba_api.stats.endpoints import LeagueGameLog
import pandas as pd
import time

def league_player_games(season: str, season_type: str = "Regular Season", timeout=90) -> pd.DataFrame:
    """One call pulls every player-game row for a season."""
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0.0.0 Safari/537.36"
        ),
        "Referer": "https://www.nba.com/",
        "Accept-Language": "en-US,en;q=0.9",
    }
    # small retry for occasional hiccups
    for attempt in range(3):
        try:
            lg = LeagueGameLog(
                season=season,                    # e.g., "2023-24"
                season_type_all_star=season_type, # "Regular Season" or "Playoffs"
                player_or_team_abbreviation="P",  # players, not teams
                timeout=timeout,
                headers=headers,
            )
            return lg.get_data_frame()
        except Exception:
            if attempt == 2:
                raise
            time.sleep(2 * (attempt + 1))

def find_bad_shooting_wins(df: pd.DataFrame) -> pd.DataFrame:
    # Normalize column names we care about
    cols = {c.upper(): c for c in df.columns}  # map upper -> actual
    def has(col): return cols.get(col, None)

    fg_pct = has("FG_PCT")
    fga    = has("FGA")
    wl     = has("WL")
    start_pos = has("START_POSITION") or has("STARTER") or has("GS")  # vary by season/endpoint

    # Base filters: team won, took at least 1 shot, shot under 10%
    mask = (df[wl] == "W") & (df[fga] > 0) & (df[fg_pct] < 0.10)

    # If a starter column exists, require it
    if start_pos is not None:
        # Some feeds use position letters; some use boolean GS; treat non-null/true as started
        started = df[start_pos].notna()
        # if it's boolean/0-1 style
        if df[start_pos].dropna().isin([0, 1, True, False]).all():
            started = df[start_pos].astype(bool)
        mask = mask & started

    out_cols = [c for c in [
        has("GAME_DATE"), has("PLAYER_ID"), has("PLAYER_NAME"), has("TEAM_ABBREVIATION"),
        fga, fg_pct, has("FGM"), has("MATCHUP"), wl, start_pos, has("MIN"), has("PTS")
    ] if c]
    return df.loc[mask, out_cols].sort_values(by=has("GAME_DATE") or out_cols[0])

# ---- Run for one season (add a loop if you want multiple) ----
season = "2023-24"
all_player_games = league_player_games(season)
result = find_bad_shooting_wins(all_player_games)

print(len(result), "games found")
print(result.head())


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=90)